In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.nn.functional as F
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
sent = pipeline('sentiment-analysis')
result = sent('what a beautiful day!')[0]
result['label'], result['score']

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


('POSITIVE', 0.9998812675476074)

In [3]:
# sentiment-analysis / text-classification / question-answering / text-generation / translation / summarization

In [5]:
text_gen = pipeline('text-generation')
result = text_gen('Alice was beginning to get very tired of sitting by her sister on the bank, ')
result

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

c:\Users\admin\anaconda3\lib\site-packages\huggingface_hub\file_download.py:138: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\admin\anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': "Alice was beginning to get very tired of sitting by her sister on the bank, \xa0which made some people worry about who would actually be there--so, she was trying to talk her off this week.\xa0 I suppose it's her parents coming"}]

In [7]:
print(result[0]['generated_text'])

Alice was beginning to get very tired of sitting by her sister on the bank,  which made some people worry about who would actually be there--so, she was trying to talk her off this week.  I suppose it's her parents coming


In [8]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased-finetuned-mrpc')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased-finetuned-mrpc')

In [10]:
print(tokenizer, '\n\n', model)

PreTrainedTokenizerFast(name_or_path='bert-base-cased-finetuned-mrpc', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}) 

 BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
             

In [11]:
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner"

In [13]:
tokens = tokenizer(input_sentence, target_sequence, return_tensors='pt')
tokens

{'input_ids': tensor([[  101,  1153, 22296,  1143,  1114,  1123, 17073,  7640,   117, 24206,
           118,  9243,   117,  1105,  4267,  1116,  4894, 26426, 21047,  1120,
          1103,  4698,  4014,  1952,   102,  1153,  1189,  1143,  4259,  1165,
          1131,  1108, 14708,  1120,  4014,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
logits = model(**tokens).logits
logits

tensor([[0.1998, 1.0848]], grad_fn=<AddmmBackward0>)

In [19]:
result = torch.softmax(logits, dim=-1)[0].tolist()
result

[0.29214248061180115, 0.7078575491905212]

In [20]:
for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 29%
yes: 71%


In [22]:
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

In [24]:
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])
X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
len(X_train), len(X_test)

(1600, 400)

In [25]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [26]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = model.to(device)

In [44]:
batch_size = 20
y_pred = []
num_batch = len(y_test) // batch_size

In [45]:
for i in range(num_batch):
    inputs = tokenizer(X_test[i*batch_size:(i+1)*batch_size], truncation=True, padding=True, return_tensors='pt')
    inputs = inputs.to(device)
    logits = model(**inputs).logits
    pred = F.softmax(logits, dim=-1)
    results = pred.cpu().detach().numpy().argmax(axis=1)
    y_pred += results.tolist()
torch.cuda.empty_cache()

In [47]:
score = sum(y_test == np.array(y_pred)) / len(y_test)
score

0.8425